<a href="https://colab.research.google.com/github/sleiph/exploracao-de-dados/blob/main/estat%C3%ADstica_nao_agrupados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estatística aplicada pra dados não agrupados

In [15]:
# o importante é importar
import numpy as np
import pandas as pd

import math

In [16]:
# quantidade de casas depois da vírgula
precisao = 4

## Declaração das funções

### Agrupando dados

In [22]:
# definindo as amplitudes
def AgruparDados(dados):
  # número de dados
  n = len(dados)

  ## tipo dos dados
  ### isDiscreta = all(isinstance(x, int) for x in dados)

  ## organizando os dados em ordem crescente
  dados_rol = sorted(dados)

  ## amplitude dos dados
  Xmin = int(dados_rol[0])
  Xmax = int(dados_rol[n-1])

  ## quantidade de classes
  if n <= 50:
    k = math.sqrt(n)
  else:
    k = int(round(math.log2(n))) + 1

  ### se tiver menor variação de dados que k, k = variação de dados
  if len(set(dados)) < k:
    k = len(set(dados))

  ## Amplitude Amostral
  AA = Xmax - Xmin

  ## Amplitude do intervalo
  h = math.ceil(AA/k)

  # criando um array com os separadores das classes
  limites = []
  ##  modos diferentes de definir as amplitudes
  ### limites = np.arange(Xmin, Xmax+1, h)
  ### limites = pd.cut(dados_rol, k, precision=precisao)

  # populando os arrays
  for r in range(Xmin, Xmax, h):
    limites.append([r, r+h, sum(x in range(r, r+h) for x in dados)])  

  return limites

### Distribuição de frequência

In [23]:
def DistrubuicaoFreqDF(limites, prec=4):
  #populando a lista

  n = len(limites)
  for i in range(len(limites)):
    limites[i].append( str(limites[i][0]) + " ⊢ " + str(limites[i][1]) )
    limites[i].append( (limites[i][0] + limites[i][1])/2 )

  # criando a tabela
  df = pd.DataFrame(
      limites,
      columns = [ 'inf', 'sup', 'fi', 'dist', 'xi' ]
  )

  # começando o índice em 1
  df.index = np.arange(1, len(df)+1)

  # adicionando valores pra cálculos
  df['fri'] = (df['fi'] / df['fi'].sum()).round(prec)
  df['fri %'] = df['fri'] * 100
  df['fac'] = df['fi'].cumsum()
  df['frac'] = (df['fac'] / df['fi'].sum()).round(prec)
  df['frac %'] = df['frac'] * 100
  df['xf'] = df['xi'] * df['fi']
  df['(xi*xi)*fi'] = (df['xi'] * df['xi']) * df['fi']

  return df

### Medidas de posição

In [24]:
# Média Aritmética
def GetMedia(dados, prec=4):
  return round( sum(dados) / len(dados), prec)

In [25]:
# Moda
def GetModa(dados, prec=4):
  if len(set(dados)) == len(dados):
    return GetPercentil(dados, .5)
  return round(max(dados, key = dados.count), prec)

In [26]:
# Percentis
def GetPercentil(dados, posicao=.5, prec=4):
  n = len(dados)
  rol = sorted(dados)
  if n%2!=0:
    return rol[ int(n*posicao) ]
  else:
    indice = int(n*posicao)
    return (rol[indice-1]+rol[indice])/2

### Medidas de dispersão

In [27]:
# variância (s²)
def GetVariancia(dados, prec=4):
  return round( sum([ (x-GetMedia(dados))**2 for x in dados ]) / len(dados), prec )

In [28]:
# desvio padrão (s)
def GetDesvio(dados, prec=4):
  return round(math.sqrt( GetVariancia(dados) ), prec)

In [29]:
# coeficiente de variação
def GetVariacaoCoef(dados, prec=4):
  return round((GetDesvio(dados) / GetMedia(dados) * 100), prec)

In [30]:
# coeficiente de assimetria de Pearson
def GetAssimetriaCoef(dados, prec=4):
  return round( (3*(GetMedia(dados)-GetPercentil(dados, .5))) / GetDesvio(dados), prec )

In [31]:
# tipo de assimetria
def GetTipoAssimetria(assimetria):
  if assimetria<0:
    return "Assimetria negativa ou à esquerda"
  elif assimetria>0:
    return "Assimetria positiva ou à direita"
  else:
    return "Assimetria nula"

In [32]:
# curtose
def GetCurtose(dados, prec=4):
  q1 = GetPercentil(dados, .25)
  q3 = GetPercentil(dados, .75)
  p90 = GetPercentil(dados, .9)
  p10 = GetPercentil(dados, .1)
  return round( (q3-q1)/(2*(p90-p10)), prec )

In [33]:
# tipo de curva
def GetTipoCurva(curtose):
  if curtose < 0.263:
    return "Leptocúrtica"
  elif curtose > 0.263:
    return "Platicúrtica"
  else:
    return "Mesocúrtica"

## Entrada de dados

In [34]:
dados_nao_agrupados = [
                       230, 232, 244, 245, 248, 249, 250, 255, 257, 260, 264, 271, 278, 280, 280,
                       280, 281, 284, 289, 292, 292, 293, 294, 294, 296, 299, 299, 299, 302, 302,
                       308, 309, 309, 310, 311, 312, 314, 315, 316, 318, 318, 320, 321, 324, 326,
                       333, 335, 335, 337, 337, 339, 341, 342, 342, 342, 348, 356, 360, 365, 369
                      ]

## Distribuição de frequência

In [ ]:
# preparar dados não agrupados
dados_agrupados = AgruparDados(dados_nao_agrupados)
dados_df = DistrubuicaoFreqDF(dados_agrupados, precisao)

dados_df

## Medidas de posição

In [ ]:
# executando os cálculos
media = GetMedia(dados_nao_agrupados, precisao)
print("media:", media)

In [ ]:
moda = GetModa(dados_nao_agrupados)
print("moda:", moda)

In [ ]:
mediana = GetPercentil(dados_nao_agrupados, .5, precisao)
print("mediana:", mediana)

In [ ]:
q1 = GetPercentil(dados_nao_agrupados, .25, precisao)
print("Q1:", q1)

In [ ]:
q3 = GetPercentil(dados_nao_agrupados, .75, precisao)
print("Q3:", q3)

## Medidas de dispersão

In [ ]:
variancia = GetVariancia(dados_nao_agrupados, precisao)
print("variância:", variancia)

In [ ]:
desvio = GetDesvio(dados_nao_agrupados, precisao)
print("desvio padrão:", desvio)

In [ ]:
coeficiente_variacao = GetVariacaoCoef(dados_nao_agrupados, precisao)
print("coeficiente de variação:", coeficiente_variacao)

In [ ]:
coeficiente_assimetria = GetAssimetriaCoef(dados_nao_agrupados, precisao)
print("coeficiente de assimetria:", coeficiente_assimetria)

In [ ]:
tipo_assimetria = GetTipoAssimetria(coeficiente_assimetria)
print("tipo de assimetria:", tipo_assimetria)

In [ ]:
curtose = GetCurtose(dados_nao_agrupados, precisao)
print("curtose:", curtose)

In [ ]:
tipo_curva = GetTipoCurva(curtose)
print("tipo de curva:", tipo_curva)